<a href="https://colab.research.google.com/github/11223344-khan/11223344-khan/blob/main/Audio_dataset_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important installations

In [ ]:
! pip install tensorflow


In [ ]:
!pip install audiomentations

In [ ]:
!pip install pydub

In [ ]:
! pip  install noisereduce

In [ ]:
!pip install ipympl

In [ ]:
!pip install ipywidgets

# Mount the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Importing necessary libraries

In [ ]:
import librosa
import os
import pickle
import joblib
import random
import numpy as np
import tensorflow as tf
import audiomentations as A
from sklearn.svm import SVC
from pydub import AudioSegment
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import noisereduce as nr
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

# Define Directory Path

In [ ]:
# Define the path to your audio files directory
audio_files_dir = '/content/gdrive/MyDrive/Audio_model/VPI Audio Clips'

# List audio files in the directory
audio_files = [os.path.join(audio_files_dir, file) for file in os.listdir(audio_files_dir)]


In [ ]:
# List the contents of the dataset folder.
os.listdir(audio_files_dir)


['Normal Samples', 'Abnormal Samples']

 code specifies a directory path 'Abnormal Samples' and lists the content of that directory.

In [ ]:
# List the content of 'Abnormal samples'
abnormal_samples_path = os.path.join(audio_files_dir, 'Abnormal Samples')
os.listdir(abnormal_samples_path)

['Video 1_2 (shoe).mp3',
 'Video 1_3 (shiny shoes).mp3',
 'Video 1_4 (seesaw).mp3',
 'Video 1_5 (see the seesaw).mp3',
 'Video 1_6 (sha, sha, sha).mp3',
 'Video 1_7 (ca, ca, ca).mp3',
 'Video 1_8 (ma, ma, ma).mp3',
 'Video 1_9 (pa, pa, pa).mp3',
 'Video 1_10 (bah, bah, bah).mp3',
 'Video 1_11 (3-9).mp3',
 'Video 1_1 (1-9).mp3',
 'Video 2_1 (put the baby in the buggy).mp3',
 'Video 2_2 (take teddy to town).mp3',
 'Video 2_3 (do it for daddy).mp3',
 'Video 2_4 (daddy).mp3',
 'Video 2_5 (baby).mp3',
 'Video 2_6 (I eat cake and ice cream).mp3',
 'Video 2_7 (go get the wagon).mp3',
 'Video 2_8 (I see the sun in the sky).mp3',
 'Video 2_9 (she went shopping).mp3',
 'Video 2_10 (I ride a choo-choo train).mp3',
 'Video 3_1 (told).mp3',
 'Video 3_2 (ted and todd).mp3',
 'Video 3_4 (don_t wear your dirty).mp3',
 'Video 3_5 (muddy).mp3',
 'Video 3_6 (muddy 2).mp3',
 'Video 3_7 (tie and hat to eat dinner).mp3',
 'Video 4_1 (it was really fun, I think it really helps me, it makes me feel great).mp3

code specifies a directory path 'Normal Samples' and lists the content of that directory.

In [ ]:
# List the cintent of 'Normal samples'
normal_samples_path = os.path.join(audio_files_dir, 'Normal Samples')
os.listdir(normal_samples_path)


['Video 1_1 (we have to have it first, then the cake).mp3',
 'Video 1_2 (shh).mp3',
 'Video 1_3 (its all of our birthdays).mp3',
 'Video 1_4 (and can I have some more material).mp3',
 'Video 1_5 (do you have a knife).mp3',
 'Video 1_6 (good).mp3',
 'Video 1_7 (here_s some pizza).mp3',
 'Video 1_8 (And I_ll put some).mp3',
 'Video 1_9 (where_s my bowl).mp3',
 'Video 1_10 (and where_s the corn sauce).mp3',
 'Video 1_11 (who, who).mp3',
 'Video 1_12 (papi are you ready).mp3',
 'Video 1_14 (fill up the gas).mp3',
 'Video 1_15 (it stops going).mp3',
 'Video 1_16 (its broke, I broke it).mp3',
 'Video 1_17 (but if its not, then we would stop running).mp3',
 'Video 1_13 (yes I_m going to give him some coffee for me and him).mp3',
 'Video 1_18 (pretend it stops running when the car is going).mp3',
 'Video 1_19 (tires).mp3',
 'video 1 (door and four).mp3',
 'video 1 (fire).mp3',
 'video 1 (four).mp3',
 'video 1 (puppies).mp3',
 'video 1 (puppy are you ready_).mp3',
 'video 1 (pizza bundt).mp3',


# Function for extracting features

In [ ]:
def extract_features(audio_file, n_mfcc=20, n_fft=1024, hop_length=512, max_length=100, noise_reduction=True):
    audio, sr = librosa.load(audio_file, sr=None)

    if noise_reduction:
        # Apply noise reduction
        reduced_noise = nr.reduce_noise(y=audio, sr=sr)

        # Use the reduced noise for feature extraction
        audio = reduced_noise

    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)

    # Pad or truncate features to have the same length (max_length)
    if mfccs.shape[1] < max_length:
        pad_width = max_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_length]

    return mfccs

# Define Pitch Augmentation

 code cell applies pitch augmentation to a list of audio files, extracts MFCC features from the augmented audio, and collects the results in the 'augmented_features' list.

In [ ]:
# Define a function to apply pitch shift
def pitch_shift(audio, semitones):
    return audio.set_frame_rate(int(audio.frame_rate * (2 ** (semitones / 12.0))))

# Data augmentation
def augment_audio(audio, max_pitch_shift=2):
    pitch_shift_amount = random.randint(1, max_pitch_shift)
    augmented_audio = pitch_shift(audio, pitch_shift_amount)
    return augmented_audio

augmented_features = []

# Iterate through the audio files and augment each one
for audio_file in audio_files:
    if audio_file.endswith(".mp3"):
        audio = AudioSegment.from_file(audio_file)
        augmented_audio = augment_audio(audio)
        mfccs = extract_features(augmented_audio, max_length=300)
        augmented_features.append(mfccs)


# Function to extract MFCC features

define a function to extract MFCC features from a collection of audio files within a specified folder. It can optionally apply data augmentation, such as pitch shifting, and returns the resulting feature as list

In [ ]:
def extract_dataset_features(folder_path, max_length=100, augment=None):
    audio_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
    features = []
    for audio_file in audio_files:
        mfccs = extract_features(audio_file, max_length=max_length)
        if augment:
            mfccs = augment(mfccs)
        features.append(mfccs)
    return features

setting maximum length

In [ ]:
max_length =200

 # Feature extarction from Normal and Abnormal Audio Samples and

In [ ]:
abnormal_features = extract_dataset_features(abnormal_samples_path, max_length=max_length)
normal_features = extract_dataset_features(normal_samples_path, max_length=max_length)


In [ ]:
abnormal_features

[array([[-805.0714   , -800.2718   , -756.1251   , ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    6.760956 ,   43.88251  , ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    6.68112  ,   57.708042 , ...,    0.       ,
            0.       ,    0.       ],
        ...,
        [   0.       ,    1.8230685,  -12.02668  , ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    1.5075717,   -6.8305435, ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    1.2210393,  -17.013405 , ...,    0.       ,
            0.       ,    0.       ]], dtype=float32),
 array([[-803.2853   , -803.2853   , -754.84924  , ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    0.       ,   39.226128 , ...,    0.       ,
            0.       ,    0.       ],
        [   0.       ,    0.       ,   51.69314  , ...,    0.       ,
            0.       ,    0.       ]

In [ ]:
normal_features

[array([[-770.8298   , -770.8298   , -765.5368   , ..., -504.7719   ,
         -435.89087  , -389.87415  ],
        [   0.       ,    0.       ,    7.461972 , ...,  163.7634   ,
          180.88815  ,  189.3056   ],
        [   0.       ,    0.       ,    7.3918886, ...,  -97.03007  ,
         -141.75404  , -163.34227  ],
        ...,
        [   0.       ,    0.       ,    2.5179262, ...,   -9.257051 ,
          -16.530655 ,  -11.666437 ],
        [   0.       ,    0.       ,    2.120369 , ...,    3.5179224,
            4.121803 ,    9.420047 ],
        [   0.       ,    0.       ,    1.7381253, ...,    7.531509 ,
           -3.7205582,    4.692148 ]], dtype=float32),
 array([[-9.3615051e+02, -9.3238312e+02, -7.9234558e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  5.1226835e+00,  1.3567245e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  4.5957723e+00,  5.6117363e+01, ...,
          0.0000000e

## Assign labels to 'normal' and 'abnormal' features
#### Creating featurer matrix X and label array y
#### '1' reprsent abnormal and '0' represent normal

In [ ]:
abnormal_labels = np.ones(len(abnormal_features))
normal_labels = np.zeros(len(normal_features))
X = np.vstack((abnormal_features, normal_features))
y = np.hstack((abnormal_labels, normal_labels))

In [ ]:
abnormal_labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

In [ ]:
normal_labels

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

# Splitting data into Training and Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

array([[[-7.59208252e+02, -7.59208252e+02, -7.31753479e+02, ...,
         -6.62804199e+02, -6.85218384e+02, -6.98765442e+02],
        [ 0.00000000e+00,  0.00000000e+00,  3.57277756e+01, ...,
         -2.39617195e+01, -1.36370049e+01,  1.46834011e+01],
        [ 0.00000000e+00,  0.00000000e+00,  3.70580368e+01, ...,
         -3.61456060e+00, -1.01650963e+01,  8.31125832e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  1.31009018e+00, ...,
          4.73269463e+00,  8.93187523e+00, -3.96058083e-01],
        [ 0.00000000e+00,  0.00000000e+00, -1.41611350e+00, ...,
         -9.30277824e+00, -5.60193968e+00, -1.03045158e+01],
        [ 0.00000000e+00,  0.00000000e+00, -7.46831656e-01, ...,
         -1.40895426e-01, -2.23640823e+00,  2.67756891e+00]],

       [[-7.58191406e+02, -7.57436829e+02, -7.22443970e+02, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.05998290e+00,  4.73457375e+01, ...,
          0.00000000e+00,  0.00000000e

In [ ]:
X_test

array([[[-8.2545874e+02, -8.2545874e+02, -7.7293768e+02, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  3.0527943e+01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  2.1011890e+01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00, -1.6266323e+01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  1.6209894e-01, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  3.2083044e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00]],

       [[-7.8725677e+02, -7.8725677e+02, -7.5053040e+02, ...,
         -7.3130811e+02, -7.4808148e+02, -7.4652002e+02],
        [ 0.0000000e+00,  0.0000000e+00,  2.3442295e+01, ...,
          5.9605202e+01,  4.2159164e+01,  3.7079926e+01],
        [ 0.0000000e+00, 

In [ ]:
y_train

array([0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0.])

In [ ]:
y_test

array([1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
       0., 0., 0., 1., 1., 1.])

# Flattening the feature matrices for Training and Testing

In [ ]:
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

In [ ]:
X_train_flattened

array([[-7.5920825e+02, -7.5920825e+02, -7.3175348e+02, ...,
        -1.4089543e-01, -2.2364082e+00,  2.6775689e+00],
       [-7.5819141e+02, -7.5743683e+02, -7.2244397e+02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-7.7743604e+02, -7.2491754e+02, -4.1876910e+02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-6.8163776e+02, -6.8163776e+02, -6.5168573e+02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-8.2607648e+02, -8.2607648e+02, -8.2020441e+02, ...,
        -1.4748606e+01, -2.8034136e+01, -3.1840534e+01],
       [-7.7244501e+02, -7.7040118e+02, -7.3336847e+02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [ ]:
X_test_flattened

array([[-825.45874 , -825.45874 , -772.9377  , ...,    0.      ,
           0.      ,    0.      ],
       [-787.2568  , -787.2568  , -750.5304  , ...,   14.294781,
           8.739563,   -4.234119],
       [-814.35333 , -814.35333 , -814.35333 , ...,    0.      ,
           0.      ,    0.      ],
       ...,
       [-713.2643  , -713.2643  , -705.2398  , ...,    0.      ,
           0.      ,    0.      ],
       [-718.6603  , -718.6603  , -718.19684 , ...,    0.      ,
           0.      ,    0.      ],
       [-786.4659  , -786.4659  , -764.5952  , ...,    0.      ,
           0.      ,    0.      ]], dtype=float32)

# Standard Scalling for Feature Data

In [ ]:
scaler = StandardScaler()

# Feature Data Scalling applied to Training and Testing

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)

In [ ]:
X_train_scaled

array([[ 0.2824782 ,  0.11761558, -0.16239679, ...,  0.01752173,
        -0.3971009 ,  0.47935757],
       [ 0.30045745,  0.1441466 , -0.08854562, ...,  0.04321731,
        -0.00872476,  0.02634686],
       [-0.03981405,  0.6311956 ,  2.3204691 , ...,  0.04321731,
        -0.00872476,  0.02634686],
       ...,
       [ 1.6540314 ,  1.2794071 ,  0.4727707 , ...,  0.04321731,
        -0.00872476,  0.02634686],
       [-0.899844  , -0.8838855 , -0.864067  , ..., -2.646536  ,
        -4.877151  , -5.360668  ],
       [ 0.04843419, -0.05002358, -0.1752083 , ...,  0.04321731,
        -0.00872476,  0.02634686]], dtype=float32)

In [ ]:
X_test_scaled

array([[-0.88892156, -0.8746335 , -0.48910597, ...,  0.04321731,
        -0.00872476,  0.02634686],
       [-0.21345824, -0.30247372, -0.31135175, ...,  2.6502054 ,
         1.5089935 , -0.6900124 ],
       [-0.69256264, -0.7083052 , -0.81765115, ...,  0.04321731,
        -0.00872476,  0.02634686],
       ...,
       [ 1.0948308 ,  0.8057292 ,  0.04793286, ...,  0.04321731,
        -0.00872476,  0.02634686],
       [ 0.9994222 ,  0.7249121 , -0.05485365, ...,  0.04321731,
        -0.00872476,  0.02634686],
       [-0.19947416, -0.29062834, -0.42292622, ...,  0.04321731,
        -0.00872476,  0.02634686]], dtype=float32)

# (SVM) Classifier Training and Evaluation

The training and testing accuracy of the SVM model is computed and displayed for the given C value, allowing for model performance assessment."

In [ ]:
param_grid = {'C': [0.001], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)
best_model = grid_search.best_estimator_

In [ ]:
y_train_pred = best_model.predict(X_train_scaled)
training_accuracy = accuracy_score(y_train, y_train_pred)
y_pred = best_model.predict(X_test_scaled)
testing_accuracy = accuracy_score(y_test, y_pred)
print(f"Best model: {best_model}")
print(f"Training Accuracy: {training_accuracy * 100:.2f}%")
print(f"Testing Accuracy: {testing_accuracy * 100:.2f}%")

Best model: SVC(C=0.001, kernel='linear')
Training Accuracy: 98.88%
Testing Accuracy: 95.65%


# Save the model

In [ ]:
with open('best_svm_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Load the model

In [ ]:
loaded_model = joblib.load('best_svm_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')

# Feature Exreaction
The feature extraction function ensures that the input audio is in the correct format and extracts relevant features for classification tasks, while the loaded model and scaler are ready for further audio analysis and classification.

In [ ]:
def extract_features(audio, n_mfcc=20, n_fft=1024, hop_length=512, max_length=200):
    # Ensure that audio data is of type numpy.ndarray and floating-point
    if not isinstance(audio, np.ndarray):
        raise ValueError("Audio data must be of type numpy.ndarray")

    if audio.dtype != np.float32:
        audio = audio.astype(np.float32) / 32768.0

    sr = 22050

    # Apply noise reduction using the noisereduce library
    reduced_audio = nr.reduce_noise(y=audio, sr=sr)

    mfccs = librosa.feature.mfcc(y=reduced_audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)

    # Pad or truncate features to have the same length (max_length)
    if mfccs.shape[1] < max_length:
        pad_width = max_length - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_length]

    return mfccs



# Audio Pitch Shifting Function
This function takes an audio signal and a parameter representing the desired pitch shift in semitones. It then adjusts the audio's frame rate to achieve the specified pitch shift, effectively altering the pitch of the audio.

In [ ]:
def pitch_shift(audio, semitones):
    return audio.set_frame_rate(int(audio.frame_rate * (2 ** (semitones / 12.0))))

# Audio Augmentation Function
This function takes an audio signal as inp and performs audio augmentation by applying a random pitch shift within a specified range.

In [ ]:
def augment_audio(audio, max_pitch_shift=2):
    pitch_shift_amount = np.random.randint(1, max_pitch_shift + 1)
    augmented_audio = pitch_shift(audio, pitch_shift_amount)
    return augmented_audio

# Audio Classification Function


In [ ]:
def classify_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)
    audio_data = np.array(audio.get_array_of_samples())
    augmented_audio = augment_audio(audio)
    mfccs = extract_features(audio_data)
    mfccs_flattened = mfccs.reshape(1, -1)
    scaled_features = loaded_scaler.transform(mfccs_flattened)
    prediction = loaded_model.predict(scaled_features)
    return prediction[0]

# Main Audio Classification Loop
 It continuously prompts the user to enter the path to an audio file they want to classify. The function then uses the classify_audio function to classify the provided audio, determining whether it is normal or abnormal

In [ ]:
def main():
    while True:
        audio_path = input("Enter the path to the audio file you want to classify: ")
        if not os.path.isfile(audio_path):
            print("File not found. Please provide a valid path.")
            continue

        prediction = classify_audio(audio_path)

        if prediction == 0:
            print("Normal voice detected.")
        else:
            print("Abnormal voice detected.")

        continue_classification = input("Do you want to classify another audio file? (yes/no): ")
        if continue_classification.lower() != 'yes':
            break

In [ ]:
if __name__ == "__main__":
    main()

Enter the path to the audio file you want to classify: /content/gdrive/MyDrive/Audio_model/VPI Audio Clips/Abnormal Samples/Facebook  (take a tire).mp3
Normal voice detected.
Do you want to classify another audio file? (yes/no): yes
Enter the path to the audio file you want to classify: /content/gdrive/MyDrive/Audio_model/VPI Audio Clips/Abnormal Samples/Facebook (susie sees the horse).mp3
Abnormal voice detected.
Do you want to classify another audio file? (yes/no): yes
Enter the path to the audio file you want to classify: /content/gdrive/MyDrive/Audio_model/VPI Audio Clips/Abnormal Samples/Video 1_2 (shoe).mp3
Normal voice detected.
Do you want to classify another audio file? (yes/no): yes
Enter the path to the audio file you want to classify: /content/gdrive/MyDrive/Audio_model/VPI Audio Clips/Abnormal Samples/Facebook (take a turtle).mp3
Abnormal voice detected.
Do you want to classify another audio file? (yes/no): yes
Enter the path to the audio file you want to classify: /conten

# Audio Classification Code


In [ ]:
# Define the function that will be called when the button is clicked
def on_button_click(b):
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    tmp_path = f'/tmp/{file_name}'
    os.rename(file_name, tmp_path)
    global uploaded_file_path
    uploaded_file_path = tmp_path
    print(f"File uploaded to {uploaded_file_path}")

    # Classify the uploaded audio
    prediction = classify_audio(uploaded_file_path)

    if prediction == 0:
        print("Normal voice detected.")
    else:
        print("Abnormal voice detected")

# Create a button widget
button = widgets.Button(description="Upload and Classify File")

# Attach the function to the button's click event
button.on_click(on_button_click)

# Display the button
display(button)


Button(description='Upload and Classify File', style=ButtonStyle())

Saving Normal.wav to Normal.wav
File uploaded to /tmp/Normal.wav
Abnormal voice detected


# Save Model to Drive

In [ ]:

# Define the paths for saving the model and scaler to Google Drive
model_path = '/content/gdrive/MyDrive/Audio_model/svm_model.pkl'
scaler_path = '/content/gdrive/MyDrive/Audio_model/scaler.pkl'

# Save the model and scaler to Google Drive
joblib.dump(best_model, model_path)
joblib.dump(scaler, scaler_path)

# Confirm that the files have been saved
print("Model and scaler saved to Google Drive.")


Model and scaler saved to Google Drive.
